# PHY3004W - Advanced Physics
## ATLAS OpenData Project 
### Jessica James [JMSJES004]

The following code was written by Kyra Kummer in 2022.

In [1]:
import uproot3 as uproot
import uproot3_methods.classes.TLorentzVector as LVepm
import uproot3_methods.classes.TVector2 as LV2
import matplotlib.pyplot as plt
from matplotlib import gridspec
import infofile 
import numpy as np
import mplhep as hep
import awkward as ak

lumi = 10 #fb^-1 "inverse femtobarns"

def get_xsec_weight(sample): 
    
    # extracts cross section weight from the info file.
    
    info = infofile.infos[sample] # open infofile
    xsec_weight = (lumi*1000*info["xsec"])/(info["sumw"]*info["red_eff"]) #*1000 to go from fb-1 to pb-1
    return xsec_weight # return cross-section weight

# Files can be downloaded from this webpage and should be placed in the same directory as the webpage
#https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/

# we start with two files: top quark pair simulation, and real data, others can be added later
files = [#"mc_361108.Ztautau.2lep.root",
#          "mc_410011.single_top_tchan.2lep.root",
#          "mc_410012.single_antitop_tchan.2lep.root",
#          "mc_410013.single_top_wtchan.2lep.root",
#          "mc_410014.single_antitop_wtchan.2lep.root",
#          "mc_410155.ttW.2lep.root",
#          "mc_410000.ttbar_lep.2lep.root", # This is the top quark pair simulation
          "data.2lep.root"               # This is the real data. Using _A version as it's much smaller,
          ]

samples = ["tt"]
#samples = ["Z tau tau", "t", "tbar", "tW", "tbarW", "ttW", "tt"]


In [2]:
# Arrays to hold events
num_events_precut = []
num_events_cut1 = []
num_events_cut2 = []
num_events_cut3 = []
num_events_cut4 = []

# Hold the histograms of the distributions for each cut that has been made. Don't forget to weight them.
cut1_hist_mll = []
cut1_hist_mbb = []
cut1_hist_mllbb = []

cut2_hist_mll = []
cut2_hist_mbb = []
cut2_hist_mllbb = []

cut3_hist_mll = []
cut3_hist_mbb = []
cut3_hist_mllbb = []

cut4_hist = []

delR_dist = []
delR_bins = np.linspace(0,7,12)

eta_dist = []
eta_bins = np.linspace(0,3,11)

delphi_dist = []
delphi_bins = np.linspace(-3.2,3.2,11)

pT_dist = []
pT_bins = np.linspace(0, 1700, 18)

# dictionary = {"key:value"} access by dictionary["key"]
chi_square_values = {}

# histogram bins:
bins_mll = np.linspace(0, 400, num=40)
bins_mbb = np.linspace(0, 400, num=40)
bins_mllbb = np.linspace(70, 700, num=40)

bins_mll_cut3 = np.linspace(0,40,num=9) # Cut 3 restricts mll to <40 GeV, so no point going to 400

data_to_tt_ratios = []

cut_titles = ["Cut 1", "Cut 2", "Cut 3", "Cut 4"]
sample_names = []

btagWP77 = 0.6459 # Constant is to do with b-tagging: classifying the b-jets.
# What does this do? 




for file in files:
    sample_name = file.split(".")[1] # file is a string, splits it at ., and takes entry at index 1.
    print("Sample name: ", sample_name) 
    
    sample_names.append(sample_name)
    print("Sample names: ", sample_names)
        
    tree = uproot.open(file)["mini"]
    
    mcWeight, SumWeights, XSection, trigM, trigE, scaleFactor_PILEUP, scaleFactor_ELE, scaleFactor_MUON, scaleFactor_LepTRIGGER, scaleFactor_BTAG, lep_type, lep_pt, lep_eta, lep_phi, lep_E, lep_charge, lep_etcone20, lep_ptcone30, jet_n, jet_pt, jet_eta, jet_phi, jet_E, jet_MV2c10,jet_trueflav, met_et, met_phi = tree.arrays(["mcWeight", "SumWeights", "XSection","trigM", "trigE","scaleFactor_PILEUP", "scaleFactor_ELE", "scaleFactor_MUON","scaleFactor_LepTRIGGER","scaleFactor_BTAG", "lep_type","lep_pt", "lep_eta","lep_phi", "lep_E", "lep_charge", "lep_etcone20", "lep_ptcone30", "jet_n", "jet_pt", "jet_eta", "jet_phi","jet_E", "jet_MV2c10", "jet_trueflav", "met_et", "met_phi"], outputtype=tuple, entrystop = 50000)
    print("File has been successfully opened!")
    
    # Create lorentz vector from transverse momentum, psuedorapidity and azimuthal
    leplv = LVepm.TLorentzVectorArray.from_ptetaphi(lep_pt, lep_eta, lep_phi, lep_E)

    # isolation for the leptons
    lep_reliso_pt = (lep_ptcone30 / lep_pt)
    lep_reliso_et = (lep_etcone20 / lep_pt)
    sum_lep_type = lep_type.sum()
    
    # Create Lorentz vector for the jets
    jetlv = LVepm.TLorentzVectorArray.from_ptetaphi(jet_pt, jet_eta, jet_phi, jet_E)
    jetlv = jetlv[jet_MV2c10.argsort()] # jet_MV2c10: output from the multivariate b-tagging algorithm of the jet
    tags = jet_pt[jet_MV2c10 > btagWP77]   #take all jets with jet_MV2c10 > btagWP77 - tags as b jets
    #Array to hold MET info
    #met_ex = met_et*np.cos(met_phi)
    #met_ey=met_et*np.sin(met_phi)
    #print(met_ex)
    #print(met_ey)
    #met=LV2.TVector2(met_ex,met_ey)
    
    trig_cut = ( (trigM==1) | (trigE==1)) # whether an event passes a single muon/electron trigger

    lep_type_cut  = (sum_lep_type == 24)    # requires an electron and muon.
    lep_iso_cut =  ((lep_reliso_pt.max() < 0.1) & (lep_reliso_et.max() < 0.1)) # making sure they are isolated.
    lept_count_cut = (leplv.counts ==2) # requires only two leptons
    lept_charge_cut = (lep_charge.sum()==0)  # requires that net zero charge
    
    # need to limit jets to two b-jets only
    num_jets_cut = (jetlv.counts == 2) # number of jets must be equal to 2
    ntag_cut = (tags.counts==2) # number of b-tagged jets must be equal to 2
    
    pre_cut = (lep_iso_cut  & lept_count_cut & lept_charge_cut & lep_type_cut & ntag_cut & num_jets_cut)
    
    #first_lep_p4 =  leplv[pre_cut, 0] # This gets the leptons from all the events without actually cutting them
    #second_lep_p4 = leplv[pre_cut, 1] # which lets us combine the del_R_cut (hopefully) but may prove difficult later?
    
    #limiting transverse momentum and pseudorapidity using values as in paper. 
    #lep_kinematics_cut  = ( (lep_pt.min() > 25000) & (lep_eta.min() >-2.5) & (lep_eta.max() < 2.5))
    
    # 1. Apply precut to the leptons and jets
    
    lep_pre = leplv[pre_cut]
    jet_pre = jetlv[pre_cut]
    met_et_pre = met_et[pre_cut]
    met_phi_pre = met_phi[pre_cut]
    
    # 2. Unpack individual leptons and jets from the pre-selected data (see 1)
    first_lep_p4 =  lep_pre[:, 0] # 4 momentum of leading lepton
    second_lep_p4 = lep_pre[:, 1] # 4 momentum of secondary lepton
    ll_p4 = first_lep_p4 + second_lep_p4 # 4 momentum of the dilepton system
    num_events_precut=len(ll_p4)
    print("Initial number of events:", num_events_precut)
    
    first_jet_p4 = jet_pre[:,0]
    second_jet_p4 = jet_pre[:,1]
    
    # 3. Get distribution of eta and pT for leading order leptons and del R
    lead_lep_pT = first_lep_p4.pt
    lead_lep_eta = first_lep_p4.eta 
    
    ll_delR = first_lep_p4.delta_r(second_lep_p4)
    
    # Weighting of events
    mcWeight = mcWeight[pre_cut]
    
    if(file.split("_")[0] == "mc"): 
        finalWeight = get_xsec_weight(sample_name)*(mcWeight)
    else:
        finalWeight = np.ones(len(mcWeight))
        
    # get a distribution of del_R
    H_delR, b = np.histogram(ll_delR, weights=np.full(len(ll_delR),finalWeight[0]), bins = delR_bins) #
    delR_dist.append(H_delR)
    
    H_eta, b = np.histogram(lead_lep_eta, weights=np.full(len(lead_lep_eta),finalWeight[0]), bins = eta_bins) 
    eta_dist.append(H_eta)
    
    H_pT, b = np.histogram(lead_lep_pT/1000, weights=np.full(len(lead_lep_pT),finalWeight[0]), bins = pT_bins) 
    pT_dist.append(H_pT)
    
    
    delR_cut = (ll_delR > 0.4)
    
    lep_kinematics_cut  = ( (lep_pre.pt.min() > 25000) & (lep_pre.eta.min() >-2.5) & (lep_pre.eta.max() < 2.5))
    
    cut_1 = (lep_kinematics_cut & delR_cut)
    
    # Now to apply cut 1 to the model and data
    first_lep_p4_1 = lep_pre[cut_1, 0] # 4 momentum of leading lepton after cut 1
    second_lep_p4_1 = lep_pre[cut_1, 1] # 4 momentum of secondary lepton after cut 1
    
    ll_p4_1 = first_lep_p4_1 + second_lep_p4_1 # 4 vectors of ll system
    
    first_jet_p4_1 = jet_pre[cut_1, 0]
    second_jet_p4_1 = jet_pre[cut_1, 1]
    
    bb_p4_1 = first_jet_p4_1 + second_jet_p4_1 # 4 vectors of bb system
    
    llbb_p4_1 = ll_p4_1 + bb_p4_1 # 4 vectors of the llbb system
    
    met_et_1=met_et_pre[cut_1]
    met_phi_1=met_phi_pre[cut_1]
    
    #print("Length of ll_p4 after cut 1:", len(ll_p4_1))
    #print("Length of bb_p4 after cut 1:", len(bb_p4_1))
    
    if len(ll_p4_1) == 0:
        num_events_cut1.append(0)    
    else:
        num_events_cut1.append(len(ll_p4_1))
    
    print("Events after cut 1:", num_events_cut1)
    
    # I want the weighting corresponding to cut 1:
    mcWeight = mcWeight[cut_1] 
        
    if(file.split("_")[0] == "mc"):
        finalWeight = get_xsec_weight(sample_name)*(mcWeight)
    else:
        finalWeight = np.ones(len(mcWeight)) 

    
    H_mll, b = np.histogram(ll_p4_1.mass/1000.0, weights=np.full(len(ll_p4_1.mass),finalWeight), bins=bins_mll) # /1000.0 to change units to GeV
    cut1_hist_mll.append(H_mll)
    
    H_mbb, b = np.histogram(bb_p4_1.mass/1000.0, weights = np.full(len(bb_p4_1.mass),finalWeight), bins=bins_mbb)
    cut1_hist_mbb.append(H_mbb)
    
    H_mllbb, b = np.histogram(llbb_p4_1.mass/1000.0, weights = np.full(len(llbb_p4_1.mass),finalWeight), bins = bins_mllbb)
    cut1_hist_mllbb.append(H_mllbb)
    
    #=====================================================================================================================
    # APPLYING CUT 2
    ll_1_delphi = first_lep_p4_1.delta_phi(second_lep_p4_1)
    #print("Delta phi: ", ll_1_delphi)
    
    H_delphi, b = np.histogram(ll_1_delphi, weights=np.full(len(ll_1_delphi),finalWeight[0]), bins = delphi_bins)
    delphi_dist.append(H_delphi) 

    delphi_cut = np.abs(ll_1_delphi) < (np.pi / 5)

    cut_2 = delphi_cut
    
    # Apply cut to leptons and jets
    first_lep_p4_2 = first_lep_p4_1[cut_2]
    second_lep_p4_2 = second_lep_p4_1[cut_2]

    ll_p4_2 = first_lep_p4_2 + second_lep_p4_2
    
    first_jet_p4_2 = first_jet_p4_1[cut_2]
    second_jet_p4_2 = second_jet_p4_1[cut_2]
    
    bb_p4_2 = first_jet_p4_2 + second_jet_p4_2
    
    llbb_p4_2 = ll_p4_2 + bb_p4_2
    
    #met_2=met_1[cut_2]
    met_et_2=met_et_1[cut_2]
    met_phi_2=met_phi_1[cut_2]

    #print(len(ll_p4_2))

    if len(ll_p4_2) == 0:
        num_events_cut2.append(0)    
    else:
        num_events_cut2.append(len(ll_p4_2))

    print("Events after cut 2:", num_events_cut2)

    ## produce a histogram of the system
    
    # I want the weighting corresponding to cut 2:
    mcWeight = mcWeight[cut_2] 
        
    if(file.split("_")[0] == "mc"):
        finalWeight = get_xsec_weight(sample_name)*(mcWeight)
    else:
        finalWeight = np.ones(len(mcWeight)) 
    
    H_mll, b = np.histogram(ll_p4_2.mass/1000.0, weights=np.full(len(ll_p4_2.mass),finalWeight), bins=bins_mll) # /1000.0 to change units to GeV
    cut2_hist_mll.append(H_mll)
    
    H_mbb, b = np.histogram(bb_p4_2.mass/1000.0, weights = np.full(len(bb_p4_2.mass),finalWeight), bins=bins_mbb)
    cut2_hist_mbb.append(H_mbb)
    
    H_mllbb, b = np.histogram(llbb_p4_2.mass/1000.0, weights = np.full(len(llbb_p4_2.mass),finalWeight), bins = bins_mllbb)
    cut2_hist_mllbb.append(H_mllbb)



    # Now applying cut 3, limit mass of dilepton system. This will be given by ll_p4_2.mass

    cut_3 = ll_p4_2.mass < (40000) # 40000 MeV = 40 GeV
    #print(cut_3)

    first_lep_p4_3 = first_lep_p4_2[cut_3]
    second_lep_p4_3 = second_lep_p4_2[cut_3]

    ll_p4_3 = first_lep_p4_3 + second_lep_p4_3
    
    first_jet_p4_3 = first_jet_p4_2[cut_3]
    second_jet_p4_3 = second_jet_p4_2[cut_3]
    
    bb_p4_3 = first_jet_p4_3 + second_jet_p4_3
    
    llbb_p4_3 = ll_p4_3 + bb_p4_3
    
    #met_3=met_2[cut_3]
    met_et_3=met_et_2[cut_3]
    met_phi_3=met_phi_2[cut_3]

    #print(len(ll_p4_3))

    if len(ll_p4_3) == 0:
        num_events_cut3.append(0)    
    else:
        num_events_cut3.append(len(ll_p4_3))

    print("Events after cut 3:", num_events_cut3)
    
    
    mcWeight = mcWeight[cut_3] 
        
    if(file.split("_")[0] == "mc"):
        finalWeight = get_xsec_weight(sample_name)*(mcWeight)
    else:
        finalWeight = np.ones(len(mcWeight)) 
    
    H_mll, b = np.histogram(ll_p4_3.mass/1000.0, weights=np.full(len(ll_p4_3.mass),finalWeight), bins=bins_mll_cut3) # /1000.0 to change units to GeV
    cut3_hist_mll.append(H_mll)
    
    H_mbb, b = np.histogram(bb_p4_3.mass/1000.0, weights = np.full(len(bb_p4_3.mass),finalWeight), bins=bins_mbb)
    cut3_hist_mbb.append(H_mbb)
    
    H_mllbb, b = np.histogram(llbb_p4_3.mass/1000.0, weights = np.full(len(llbb_p4_3.mass),finalWeight), bins = bins_mllbb)
    cut3_hist_mllbb.append(H_mllbb)


    ## Cut 4 requires that the transverse mass of the llbbvv system is known.

Sample name:  2lep
Sample names:  ['2lep']
File has been successfully opened!
Initial number of events: 56
Events after cut 1: [42]
Events after cut 2: [8]
Events after cut 3: [3]


## Kinematic Reconstruction
We want to create an 18 column array with the tt events:
- 2x lepton 4-momentum
- 2x B-jet 4-momentum
- W mass

In [3]:
system_array = [first_lep_p4_3, second_lep_p4_3, first_jet_p4_3, second_jet_p4_3, met_et_3, met_phi_3]
len(system_array)

6

In [4]:
# system_array[column][row]
len(system_array[5])


3

Working from C++ code written by James Mitchley

In [5]:
def setSolutions(num, a, b):
    m_v1=a
    m_v2=b
    m_solutions=num
    
def setSolutions(num):
    m_solutions=num
    
def getNumSolutions():
    return m_solutions

def getv1():
    return m_v1

def getv2():
    return m_v2

def solveForNeutrinoEta(lepton, bjet, topmass, index, neutrinos):
    m_wmass = 80400
    Wmass2 = m_wmass * m_wmass
    m_bmass = 4180
    bmass = m_bmass

    Elprime = lepton.E() * neutrinos[index][2] - lepton.Pz() * neutrinos[index][1]
    Ebprime = bJet.E() * neutrinos[index][2] - bJet.Pz() * neutrinos[index][1]

    A = (lepton.Py() * Ebprime - bJet.Py() * Elprime) / (bJet.Px() * Elprime - lepton.Px() * Ebprime)
    B = (Elprime * (topMass * topMass - Wmass2 - bmass * bmass - 2. * lepton * bJet) - Ebprime * Wmass2) /(2. * (lepton.Px() * Ebprime - bJet.Px() * Elprime))

    par1 = (lepton.Px() * A + lepton.Py()) / Elprime
    C = A * A + 1. - par1 * par1
    par2 = (Wmass2 / 2. + lepton.Px() * B) / Elprime
    D = 2. * (A * B - par2 * par1)
    F = B * B - par2 * par2
    det = D * D - 4. * C * F

    sol.setSolutions(0)

    if det>0:
    
        tmp = np.sqrt(det) / (2. * C)
        py1 = -D / (2. * C) + tmp
        py2 = -D / (2. * C) - tmp
        px1 = A * py1 + B
        px2 = A * py2 + B
        pT2_1 = px1 * px1 + py1 * py1
        pT2_2 = px2 * px2 + py2 * py2
        pz1 = np.sqrt(pT2_1) * neutrinos[index][1]
        pz2 = np.sqrt(pT2_2) * neutrinos[index][1]

        a1(px1, py1, pz1, sqrt(pT2_1 + pz1 * pz1))
        a2(px2, py2, pz2, sqrt(pT2_2 + pz2 * pz2))

        sol.setSolutions(2, a1, a2);

    return sol

def neutrino_weight(neutrino1, neutrino2, met_ex, met_ey):
    sigmax = 55643.82942235848
    sigmay = 54224.08180662581
    dx = met_ex - neutrino1.Px() - neutrino2.Px()
    dy = met_ey - neutrino1.Py() - neutrino2.Py()

    return exp(-dx * dx / (2. * sigmax * sigmax) - dy * dy / (2. * sigmay * sigmay))




In [6]:
# Create neutrino array
neutrinos = np.zeros((2000,3))
etaStep = 0.2
index = 0 
eta=-5

while eta<5.0001:
    neutrinos[index][0]=eta
    neutrinos[index][1]=np.sinh(eta)
    neutrinos[index][2]=np.cosh(eta)
    index +=1
    eta+=etaStep
    
etaSize=index
nEvents = num_events_cut3[0]

# Arrays to hold neutrino data
v0_pt = np.array(nEvents)
v0_eta = np.array(nEvents)
v0_phi = np.array(nEvents)
v0_m = np.array(nEvents)
v1_pt = np.array(nEvents)
v1_eta = np.array(nEvents)
v1_phi = np.array(nEvents)
v1_m = np.array(nEvents)

for i in range(0, nEvents):
    # Lepton and Jet 4-Vectors
    l0 = system_array[0][i]
    print(type(l0))
    l1 = system_array[1][i]
    b0 = system_array[2][i]
    b1 = system_array[3][i]
    
    # We need MET in x- and y-directions, currently in polar coordinates
    met_ex = met_et*np.cos(met_phi)
    met_ey = met_et*np.sin(met_phi)
    
    max_weight=0
    topMass=172500
    
    # Attempt to solve eqns for different values of eta
    for j in range(0, etaSize):
        
        # Need to define solveForNeutrinoEta
        ans1= solveForNeutrinoEta(l0, b0, topMass, j, neutrinos)
        ans2= solveForNeutrinoEta(l1, b1, topMass, j, neutrinos)
        ans3= solveForNeutrinoEta(l0, b1, topMass, j, neutrinos)
        ans4= solveForNeutrinoEta(l1, b0, topMass, j, neutrinos)
        
        if (ans1.getNumSolutions()>0 and ans2.getNumSolutions()>0):
            sol1 = neutrino_weight(ans1.getv1(), ans2.getv1(), met_ex, met_ey)
            sol2 = neutrino_weight(ans1.getv1(), ans2.getv2(), met_ex, met_ey)
            sol3 = neutrino_weight(ans1.getv2(), ans2.getv1(), met_ex, met_ey)
            sol4 = neutrino_weight(ans1.getv2(), ans2.getv2(), met_ex, met_ey)
            
        if (ans3.getNumSolutions()>0 and ans4.getNumSolutions()>0):
            sol5 = neutrino_weight(ans3.getv1(), ans4.getv1(), met_ex, met_ey)
            sol6 = neutrino_weight(ans3.getv1(), ans4.getv2(), met_ex, met_ey)
            sol7 = neutrino_weight(ans3.getv2(), ans4.getv1(), met_ex, met_ey)
            sol8 = neutrino_weight(ans3.getv2(), ans4.getv2(), met_ex, met_ey)
            
        # Save the solution with the highest weight
        if ans1.getNumSolutions() > 0:
            weight=sol1
            v0 = ans1.getv1()
            v2 = ans2.getv2()
            
        if (sol2 > weight):
                weight = sol2
                v0 = ans1.getv1()
                v1 = ans2.getv2()

        if (sol3 > weight):
            weight = sol3
            v0 = ans1.getv2()
            v1 = ans2.getv1()

        if (sol4 > weight):
            weight = sol4
            v0 = ans2.getv2()
            v1 = ans2.getv2()

        if (sol5 > weight):
            weight = sol5
            v0 = ans3.getv1()
            v1 = ans4.getv1()

        if (sol6 > weight):
            weight = sol6
            v0 = ans3.getv1()
            v1 = ans4.getv2()

        if (sol7 > weight):
            weight = sol7
            v0 = ans3.getv2()
            v1 = ans4.getv1()

        if (sol8 > weight):
            weight = sol8
            v0 = ans3.getv2()
            v1 = ans4.getv2()

        #Save solution with largest weight per neutrino eta
        if (weight > max_weight):
            max_weight = weight
            bv0 = v0
            bv1 = v1
        
    # Save solution to neutrino array
    v0_pt[i] = bv0.Pt();
    v0_eta[i] = bv0.Eta();
    v0_phi[i] = bv0.Phi();
    v0_m[i] = bv0.M();
    v1_pt[i] = bv1.Pt();
    v1_eta[i] = bv1.Eta();
    v1_phi[i] = bv1.Phi();
    v1_m[i] = bv1.M();

# Save solutions to a .txt file
f=file.open("neutrinos.txt", w)
for i in range(0,nEvents):
    f.write(str(v0_pt[i])+' '+str(v0_eta[i])+" "+str(v0_phi[i])+" "+str(v0_m[i])+" "+ str(v1_pt[i]) + " " + str(v1_eta[i]) + " " + str(v1_phi[i]) + " " + str(v1_m[i]) + "\n")
f.close()

<class 'uproot3_methods.classes.TLorentzVector.TLorentzVector'>


TypeError: 'float' object is not callable